In [1]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.optimizers import Adam

Using TensorFlow backend.
F:\conda_3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\conda_3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\conda_3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\conda_3.7\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [2]:
import cv2

def read_image(filename, byteorder='>'):
    
    #first we read the image, as a raw file to the buffer
    #with open(filename, 'rb') as f:
     #   buffer = f.read()
        
    img = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(150,150))
    
    #using regex, we extract the header, width, height and maxval of the image
    #header, width, height, maxval = re.search(
      #   b"(^P5\s(?:\s*#.*[\r\n])*"
      #  b"(\d+)\s(?:\s*#.*[\r\n])*"
      #  b"(\d+)\s(?:\s*#.*[\r\n])*"
      #  b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    
    #then we convert the image to numpy array using np.frombuffer which interprets buffer as one dimensional array
    
    return np.array(img) 

In [3]:
size = 2
total_sample_size = 1000


def get_data(size, total_sample_size):
    #read the image
    image = read_image('Faces/s' + str(1) + '/' + str(1) + '.jpg', 'rw+')
    #reduce the size
    image = image[::size, ::size]
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0
    
    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2]) # 2 is for pairs
    y_genuine = np.zeros([total_sample_size, 1])
    
    for i in range(8):
        for j in range(int(total_sample_size/8)):
            ind1 = 0
            ind2 = 0
            
            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(10)
                ind2 = np.random.randint(10)
                
#             print(ind1,ind2)
            
            # read the two images
            img1 = read_image('Faces/s' + str(i+1) + '/' + str(ind1 + 1) + '.jpg', 'rw+')
            img2 = read_image('Faces/s' + str(i+1) + '/' + str(ind2 + 1) + '.jpg', 'rw+')
            
            #reduce the size
            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]
            
            #store the images to the initialized numpy array
            x_geuine_pair[count, 0, 0, :, :] = img1
            x_geuine_pair[count, 1, 0, :, :] = img2
            
            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
    y_imposite = np.zeros([total_sample_size, 1])
    
    for i in range(int(total_sample_size/10)):
        for j in range(10):
            
            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(8)
                ind2 = np.random.randint(8)
                if ind1 != ind2:
                    break
                    
            img1 = read_image('Faces/s' + str(ind1+1) + '/' + str(j + 1) + '.jpg', 'rw+')
            img2 = read_image('Faces/s' + str(ind2+1) + '/' + str(j + 1) + '.jpg', 'rw+')

            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, 0, :, :] = img1
            x_imposite_pair[count, 1, 0, :, :] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1
            
    #now, concatenate, genuine pairs and imposite pair to get the whole data
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [4]:
X, Y = get_data(size, total_sample_size)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2,random_state=42)

In [6]:
def build_base_network(input_shape):
    
    seq = Sequential()
    
    nb_filter = [6, 12]
    kernel_size = 3
    
    
    #convolutional layer 1
    seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape,
                          border_mode='valid', dim_ordering='th'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2))) 
    seq.add(Dropout(.25))
    
    #convolutional layer 2
    seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size, border_mode='valid', dim_ordering='th'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2), dim_ordering='th')) 
    seq.add(Dropout(.25))

    #flatten 
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(50, activation='relu'))
    return seq

In [7]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

F:\conda_3.7\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (3, 3), input_shape=(1, 75, 75..., padding="valid", data_format="channels_first")`
  # This is added back by InteractiveShellApp.init_path()
F:\conda_3.7\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(12, (3, 3), padding="valid", data_format="channels_first")`
F:\conda_3.7\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`


In [8]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [9]:
epochs = 13
rms = RMSprop()

model = Model(input=[img_a, img_b], output=distance)

F:\conda_3.7\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("la...)`
  after removing the cwd from sys.path.


In [10]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

model.compile(optimizer=Adam(lr=0.001),loss=contrastive_loss)

In [15]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1] 

model.fit([img_1, img_2], y_train, validation_split=.2, batch_size=128, verbose=1, nb_epoch=10)

F:\conda_3.7\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.


Train on 1280 samples, validate on 320 samples
Epoch 1/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.1189 - val_loss: 0.1887
Epoch 2/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.1101 - val_loss: 0.1548
Epoch 3/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.1000 - val_loss: 0.1675
Epoch 4/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.1036 - val_loss: 0.1658
Epoch 5/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.0979 - val_loss: 0.1402
Epoch 6/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.0941 - val_loss: 0.1377
Epoch 7/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.0851 - val_loss: 0.1212
Epoch 8/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.0822 - val_loss: 0.1296
Epoch 9/10
1280/1280 [==============================] - 9s 7ms/step - loss: 0.0785 - val_loss: 0.1148
Epoch 10/10
1280/1280 [============

In [16]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])

In [17]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

In [18]:
compute_accuracy(y_test,pred)

0.4990375